In [1]:
from torch_geometric.datasets import EllipticBitcoinDataset

# Load the dataset
root = 'EllipticBitcoin'
dataset = EllipticBitcoinDataset(root)
data = dataset[0]

In [2]:
data.x.shape

torch.Size([203769, 165])

In [4]:
data.x

tensor([[-0.1715, -0.1847, -1.2014,  ..., -0.0975, -0.1206, -0.1198],
        [-0.1715, -0.1847, -1.2014,  ..., -0.0975, -0.1206, -0.1198],
        [-0.1721, -0.1847, -1.2014,  ..., -0.1837, -0.1206, -0.1198],
        ...,
        [-0.1720, -0.0782,  1.0186,  ..., -0.0975, -0.1206, -0.1198],
        [-0.1728, -0.1766,  1.0186,  ..., -0.1406,  1.5197,  1.5214],
        [-0.0120, -0.1323,  0.4636,  ..., -0.1406,  1.5197,  1.5214]])

In [5]:
data.edge_index

tensor([[     0,      2,      4,  ..., 201921, 201480, 201954],
        [     1,      3,      5,  ..., 202042, 201368, 201756]])

: 

In [4]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "k59D5ftw^N^WUm"))

cypher_query = '''
UNWIND $items AS item
MERGE (a:Transaction {id: item.id})
SET a.features = item.features, a.label = item.label
WITH a, item.edges AS edges
UNWIND edges AS edge
MERGE (b:Transaction {id: edge.dst_id})
MERGE (a)-[:PAYMENT]->(b)
'''

def create_graph(tx, batch):
    tx.run(cypher_query, items=batch)

# Define batch size
batch_size = 1000

# Upload nodes and edges in batches
with driver.session() as session:
    for i in range(0, data.num_nodes, batch_size):
        batch = []
        for node_id in range(i, min(i + batch_size, data.num_nodes)):
            node = {"id": node_id, "features": data.x[node_id].tolist(), "label": data.y[node_id].item(), "edges": []}
            node_edges = [{"src_id": node_id, "dst_id": dst.item()} for dst in data.edge_index[1, data.edge_index[0] == node_id]]
            node["edges"] = node_edges
            batch.append(node)
        session.execute_write(create_graph, batch)

driver.close()

In [6]:
import requests

# Define batch size
batch_size = 1000

# Upload nodes and edges in batches
node_url = "http://localhost:5004/api/nodes/batch"
edge_url = "http://localhost:5004/api/edges/batch"
headers = {"Content-Type": "application/json"}

# Prepare 1000 nodes from the dataset
nodes = []
for i in range(1000):
    node = {
        'labels': ['BitcoinTransaction'],
        'time_step': int(data.x[i, 0]),
        'tx_features': data.x[i, 1:].tolist()
    }
    nodes.append(node)
    
# Send the POST request to the server
response = requests.post(node_url, json=nodes)

# Check if the request was successful
if response.status_code == 201:
    # Print the returned node IDs
    print("Node IDs:")
    for node in response.json():
        print(node['id'])  # Adjust according to your server response structure
else:
    print(f"Failed to create nodes: {response.json()}")

Node IDs:
bit_Y8S8atHk7kmt8v2N
bit_PAfkxsaDXWxosvSb
bit_t7Y94p2Nmr6HhKk6
bit_pufdApZJDZrZpzdk
bit_9CMyiN3gPk4VwMRq
bit_wgHMGYLHMZzcGJcc
bit_eDiHi3NMgAer4WWB
bit_H7R4gQefrV7rLpid
bit_XEmvYehHezR9UQzt
bit_zgoco6vkyLMeVrgL
bit_DhdXABVPvVYg2Zr9
bit_oYmUrDabAtpTYWd8
bit_LZSDaYJGVxQLzMao
bit_4Zoh7SiZMGvKG79p
bit_7HBgyeuRyUDNy2yx
bit_FtVaZNTDdfVwSSQF
bit_Axra6EyyBsZ23moc
bit_YYuGbW8fW5jGfusV
bit_RsVft7YPoKagdHNN
bit_f4Q2jBxiqFx8dixT
bit_F7u7xwTJHfy8ZvSQ
bit_h42FzJ6ThzTK5xju
bit_D8faZrAsDSNGwBLZ
bit_m5gx6vgMNeRAHr5u
bit_eubN9LYP8j7zfVSi
bit_z8xiG2BGU94q5K9z
bit_cQ7EaJgoeQaf3dfh
bit_5JyQseQpLz3oaWsx
bit_yw4gHuEvV6qGYxNs
bit_xqzVR68hsDFNQhDP
bit_LmL4tGZiPH6RKecD
bit_uKNgsstxxRKfyFd8
bit_yTUcnKCFHXvCu4L8
bit_opMVnZJwRa4MxfYG
bit_nXE3bMkPj4U3T7uE
bit_Bm5WDAtMgMtPGX5t
bit_JUV9dtdrbD4f2SDV
bit_G4kUJfdprBjyyW7L
bit_HsjeRTYBw6e94s6T
bit_Ei5C8p6aE4mJyuka
bit_G7HTgfAz6MybjvwR
bit_MZu2zGFxEeZrK2kE
bit_deeiHzx5xTFboXyg
bit_LJSPfkcX8rLESB9s
bit_PmHLhfpP6fS5Lje7
bit_GjArYNypP6d87jpv
bit_y3XLAcSXVwz7uGSo
bit

In [20]:
shortuuid.ShortUUID().random(length=16)

'bqjXN8RjXYemoSG4'

In [23]:
"1".lower()

'1'